In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from acquire import get_cohort_data, get_cohort_information_data
from prepare import split_by_program
from wrangle import wrangle_cohort_data

**Definitions**
- php - PHP Full Stack Web Development
- java - Java Full Stack Web Development
- ds - Data Science
- fe - Front End Web Development

## Acquire

In [2]:
# Created an acquire.py file that runs a SQL query to join the databases from the codeup library, and created a
# pandas dataframe with that query, after that I saved it to a csv, and have to function check for the csv.
df = get_cohort_data()

In [3]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        847330 non-null  object 
 1   time        847330 non-null  object 
 2   path        847329 non-null  object 
 3   user_id     847330 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          847330 non-null  object 
 6   id          72405 non-null   float64
 7   name        72405 non-null   object 
 8   slack       72405 non-null   object 
 9   start_date  72405 non-null   object 
 10  end_date    72405 non-null   object 
 11  created_at  72405 non-null   object 
 12  updated_at  72405 non-null   object 
 13  deleted_at  0 non-null       float64
 14  program_id  72405 non-null   float64
dtypes: float64(4), int64(1), object(10)
memory usage: 103.4+ MB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,847330.0,456.707344,250.734201,1.0,263.0,476.0,648.0,981.0
cohort_id,847330.0,48.501049,32.795482,1.0,28.0,33.0,57.0,139.0
id,72405.0,30.015248,35.619910,1.0,11.0,12.0,53.0,139.0
deleted_at,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
program_id,72405.0,1.552876,0.662231,1.0,1.0,1.0,2.0,4.0


In [6]:
df.isnull().sum()

date               0
time               0
path               1
user_id            0
cohort_id          0
ip                 0
id            774925
name          774925
slack         774925
start_date    774925
end_date      774925
created_at    774925
updated_at    774925
deleted_at    847330
program_id    774925
dtype: int64

In [7]:
df.shape

(847330, 15)

In [8]:
df_info = get_cohort_information_data()

In [9]:
df_info.head()

,name,start_date,end_date,program_id
cohort_id,,,,
1,Arches,2014-02-04,2014-04-22,1
2,Badlands,2014-06-04,2014-08-22,1
3,Carlsbad,2014-09-04,2014-11-05,1
4,Denali,2014-10-20,2015-01-18,1
5,Everglades,2014-11-18,2015-02-24,1


In [10]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 1 to 139
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        52 non-null     object
 1   start_date  52 non-null     object
 2   end_date    52 non-null     object
 3   program_id  52 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 2.0+ KB


In [11]:
df_info.describe().T

,count,mean,std,min,25%,50%,75%,max
program_id,52.0,1.923077,0.736876,1.0,1.0,2.0,2.0,4.0


In [12]:
df_info.isnull().sum()

name          0
start_date    0
end_date      0
program_id    0
dtype: int64

In [13]:
df_info.shape

(52, 4)

## Prepare

**Prepare summary**

##### List of things my clean_cohort_data function does.
- Combined the date and time column into a datetime, and converted the datatype to datetime.
- Placed my new datetime column to the index for later use.
- Replaced the program_id with its corresponding program name and subdomain.
- Dropped unnecessary columns 'date','time','deleted_at','program_id','id'.
- Renamed 'name' to 'cohort_name' for my personal readability.
- Created a dictionary to map cohort name to cohort id.
- Filled nulls in cohort_id.

## Wrangle

**Size before wrangle - (847330, 15),(52, 4)**

**Size after wrangle - (847329, 12)**

**This is the cleaned dataframe and will be used to answer the questions given in the email by zach**

In [14]:
df = wrangle_cohort_data()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 801475 entries, 2018-01-26 09:56:02 to 2021-04-21 12:32:01
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   path               801475 non-null  object
 1   user_id            801475 non-null  int64 
 2   cohort_id          801475 non-null  int64 
 3   ip                 801475 non-null  object
 4   cohort_name        801475 non-null  object
 5   start_date         801475 non-null  object
 6   end_date           801475 non-null  object
 7   program_id         801475 non-null  int64 
 8   count_helper       801475 non-null  int64 
 9   program_name       801475 non-null  object
 10  program_subdomain  801475 non-null  object
 11  slack              801475 non-null  object
dtypes: int64(4), object(8)
memory usage: 79.5+ MB


In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,801475.0,457.292479,249.980598,1.0,264.0,476.0,648.0,981.0
cohort_id,801475.0,48.558252,32.833081,1.0,28.0,33.0,57.0,139.0
program_id,801475.0,2.082590,0.384466,1.0,2.0,2.0,2.0,4.0
count_helper,801475.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [17]:
df.isnull().sum()

path                 0
user_id              0
cohort_id            0
ip                   0
cohort_name          0
start_date           0
end_date             0
program_id           0
count_helper         0
program_name         0
program_subdomain    0
slack                0
dtype: int64

In [18]:
df.shape

(801475, 12)

In [19]:
df.head()

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,count_helper,program_name,program_subdomain,slack
datetime,,,,,,,,,,,,
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,1,PHP Full Stack Web Development,php,#hampton
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,1,PHP Full Stack Web Development,php,#hampton
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,1,PHP Full Stack Web Development,php,#hampton
2018-01-26 10:40:15,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,1,PHP Full Stack Web Development,php,#hampton
2018-01-26 11:26:13,java-i,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,1,PHP Full Stack Web Development,php,#hampton


### Zach's questions in the email:

### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [20]:
php_df,java_df,ds_df,fe_df = split_by_program(df)

In [21]:
php_df.groupby('cohort_name').path.value_counts(normalize = True).sort_values(ascending = False)

cohort_name  path                                  
Denali       prework/databases                         0.333333
             mkdocs/search_index.json                  0.333333
             prework/versioning/github                 0.333333
Franklin     javascript-ii/es6                         0.147059
Lassen       index.html                                0.092687
                                                         ...   
             appendix/extra-challenges/css-practice    0.000106
             appendix/extra-challenges                 0.000106
             appendix/cli-git-overview                 0.000106
             appendix/angular/templating               0.000106
             spring/fundamentals/integration-tests     0.000106
Name: path, Length: 2555, dtype: float64